 # Надо:
* Считать данные из текстового файла
* Переделать в numpy array(2 размерности)
* **ОБЯЗАТЕЛЬНО** нагенерить данных для обучения
* Разбить на выборки
* Обучить модель
* Перевести предсказания в пары

In [59]:
import numpy as np
import pandas as ps

In [12]:
dt = open('example.txt', 'r')
dt = dt.readlines()
dat = [] #Будущие данные о пользователях
for usr in dt:
    dat.append(usr.split(',')) #ВАЖНО! разделитель данных -- ЗАПЯТАЯ

''' Данные о пользователе: [id, nickname, room_number, direction, (sports...)]
                           [0,  kllllevo, 13,           1,         0,1,0,0]
'''

' Данные о пользователе: [id, nickname, room_number, direction, (sports...)]\n                           [0,  kllllevo, 13,           1,         0,1,0,0]\n'

In [13]:
dat

[['1', 'isus in courtyar', '34', '1', '1', '1', '0\n'],
 ['2', 'fubar or_Alah', '12', '3', '0', '1', '1\n'],
 ['3', 'elevenponies', '12', '2', '1', '0', '1\n'],
 ['4', 'environment of home', '10', '2', '1', '1', '0\n'],
 ['5', 'furie', '9', '1', '0', '0', '1\n'],
 ['6', 'europe+', '34', '3', '0', '0', '0\n'],
 ['7', 'round of Earth', '35', '3', '1', '0', '0\n'],
 ['8', 'jgeek', '10', '1', '0', '1', '1\n'],
 ['9', 'whatisit', '7', '2', '0', '0', '1\n'],
 ['10', 'why is this harder then usual project?', '1', '4', '1', '1', '0\n']]

In [18]:
data = {}
for usr in dat:
    data[usr[0]] = list(usr[1:])

''' Данные о пользователях: {'id':[Данные о пользователе, кроме id]} 
'''

" Данные о пользователях: {'id':[Данные о пользователе, кроме id]} \n"

In [19]:
data

{'1': ['isus in courtyar', '34', '1', '1', '1', '0\n'],
 '10': ['why is this harder then usual project?', '1', '4', '1', '1', '0\n'],
 '2': ['fubar or_Alah', '12', '3', '0', '1', '1\n'],
 '3': ['elevenponies', '12', '2', '1', '0', '1\n'],
 '4': ['environment of home', '10', '2', '1', '1', '0\n'],
 '5': ['furie', '9', '1', '0', '0', '1\n'],
 '6': ['europe+', '34', '3', '0', '0', '0\n'],
 '7': ['round of Earth', '35', '3', '1', '0', '0\n'],
 '8': ['jgeek', '10', '1', '0', '1', '1\n'],
 '9': ['whatisit', '7', '2', '0', '0', '1\n']}

In [24]:
drata = []
for i in list(data.keys()):
    a = list(data.keys())
    a.remove(i)
    drata.append([val for val in a])

''' Данные вида [another users id] -- this for user with id=0
                [another users id] -- for id=1
                [another users id] -- for id=2
'''
drata

[['2', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['1', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['1', '2', '4', '5', '6', '7', '8', '9', '10'],
 ['1', '2', '3', '5', '6', '7', '8', '9', '10'],
 ['1', '2', '3', '4', '6', '7', '8', '9', '10'],
 ['1', '2', '3', '4', '5', '7', '8', '9', '10'],
 ['1', '2', '3', '4', '5', '6', '8', '9', '10'],
 ['1', '2', '3', '4', '5', '6', '7', '9', '10'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '10'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']]

In [89]:
final_data = []
nmb = 1 # id юзера(см. ниже)
for lst in drata:
    with_top = {}
    top = []
    for u in lst:
        u_data = np.array(data[u][1:])# Вся дата того, кого проверяем, кроме nickname
        usr_data = np.array(data[str(nmb)][1:])# Вся дата того, ДЛЯ кого проверяем, кроме nickname
        with_top[u] = len(np.where(u_data == usr_data)[0])
        
    #for znach in sorted(val for key,val in with_top.items()):
    #   for key,val in with_top.items():
    #        if val==znach:
    #            top.append(key)
    #            
    #final_data.append(top)

In [90]:
final_data[0]

['9', '2', '2', '2', '6', '6', '6', '4', '4']